In [1]:
import numpy as np 
import pandas as pd
#import polars as pl
import sys
from astropy.io import fits
import sqlite3
import plotly.graph_objects as go
from galpy.orbit import Orbit
from galpy.potential import MWPotential2014
from astropy.coordinates import SkyCoord
import astropy.units as u
import astropy.units
sys.path.append('../../dash_gaia_orbits/')
from sklearn.neighbors import KernelDensity

import matplotlib.pyplot as plt
%matplotlib widget
plt.style.use('dark_background')

import colormaps as cmaps

import plotly.graph_objects as go

# Package imports
from dash_gaia_orbits import StarClusterData, StarClusterCollection, StarClusters3DPlotter

A new version of galpy (1.9.2) is available, please upgrade using pip/conda/... to get the latest features and bug fixes!


In [2]:
conn = sqlite3.connect('/Users/cam/Desktop/astro_research/radcliffe/cluster_database/star_clusters.db')

query = """
SELECT * FROM star_clusters
WHERE source = 'Hunt+2023'
AND U not NULL AND V not NULL AND W not NULL
AND U_err not NULL AND V_err not NULL AND W_err not NULL
AND U_err < 5 AND V_err < 5 AND W_err < 5
AND U_err != 0 AND V_err != 0 AND W_err != 0
AND x between -2000 and 2000 and y between -2000 and 2000 and z between -300 and 300
AND n_stars_rv >= 3
AND age_myr <= 150 or name = "Melotte_25" or name = "Melotte_22"
"""

pleiades_hyades = """ SELECT * FROM star_clusters WHERE name = 'Melotte_22' or name = 'Melotte_25' or name = 'NGC_2516' or name = 'NGC 2632'"""


df = pd.read_sql_query(query, conn)

In [3]:
clusters = StarClusterData(df, data_name = 'Hunt+2023', min_size = 0, max_size = 6, color = 'black', opacity = 1., marker_style = 'circle')
clusters.integrate_orbits(time = np.arange(0., -100., -1.))
df_int = clusters.df_int
#sc = StarClusterCollection([clusters])

  0%|          | 0/784 [00:00<?, ?it/s]

In [4]:
def kde2D(x, y, bandwidth, xbins=100j, ybins=100j, **kwargs): 
    """Build 2D kernel density estimate (KDE)."""

    # create grid of sample locations (default: 100x100)
    xx, yy = np.mgrid[x.min():x.max():xbins, 
                      y.min():y.max():ybins]

    xy_sample = np.vstack([yy.ravel(), xx.ravel()]).T
    xy_train  = np.vstack([y, x]).T

    kde_skl = KernelDensity(bandwidth=bandwidth, **kwargs)
    kde_skl.fit(xy_train)

    # score_samples() returns the log-likelihood of the samples
    z = np.exp(kde_skl.score_samples(xy_sample))
    return xx, yy, np.reshape(z, xx.shape)

In [13]:
def make_plot(img, title, extent):
    plt.figure(figsize = (6,6))
    plt.imshow(img.T, cmap = cmaps.ice, extent = extent, origin = 'lower', vmax = 5.)
    plt.xlim(-2000, 2000)
    plt.ylim(-2000, 2000)
    plt.savefig('/Users/cam/Desktop/astro_research/radcliffe/cluster_tb_plots/{}.jpeg'.format(title), dpi = 300)
    plt.close()

In [ ]:
times = np.arange(0., -61., -1.)

for t in times:
    df_t  = df_int.loc[df_int.time == t]
    df_t = df_t.loc[df_t.age_myr.between(np.abs(t) - 15, np.abs(t) + 15)]
    x, y, z = df_t.x.values, df_t.y.values, df_t.z.values
    extent = [x.min(), x.max(), y.min(), y.max()]
    xx, yy, zz = kde2D(x, y, 100, kernel = 'epanechnikov')
    xx2, yy2, zz2 = kde2D(x, z, 100, kernel = 'epanechnikov')
    img = (zz/zz2) - 1
    make_plot(img, t, extent)

/var/folders/1d/0pb_hyfs6ljgsf6pclkykfy40000gn/T/ipykernel_88534/3078499929.py:10: RuntimeWarning: divide by zero encountered in divide
  img = (zz/zz2) - 1

/var/folders/1d/0pb_hyfs6ljgsf6pclkykfy40000gn/T/ipykernel_88534/3078499929.py:10: RuntimeWarning: invalid value encountered in divide
  img = (zz/zz2) - 1

/var/folders/1d/0pb_hyfs6ljgsf6pclkykfy40000gn/T/ipykernel_88534/3078499929.py:10: RuntimeWarning: divide by zero encountered in divide
  img = (zz/zz2) - 1

/var/folders/1d/0pb_hyfs6ljgsf6pclkykfy40000gn/T/ipykernel_88534/3078499929.py:10: RuntimeWarning: invalid value encountered in divide
  img = (zz/zz2) - 1

/var/folders/1d/0pb_hyfs6ljgsf6pclkykfy40000gn/T/ipykernel_88534/3078499929.py:10: RuntimeWarning: divide by zero encountered in divide
  img = (zz/zz2) - 1

/var/folders/1d/0pb_hyfs6ljgsf6pclkykfy40000gn/T/ipykernel_88534/3078499929.py:10: RuntimeWarning: invalid value encountered in divide
  img = (zz/zz2) - 1

/var/folders/1d/0pb_hyfs6ljgsf6pclkykfy40000gn/T/ipyker

No such comm: 1aedc0dc276b4347bd4bf6fa2c88c48d
No such comm: 1aedc0dc276b4347bd4bf6fa2c88c48d
No such comm: 1aedc0dc276b4347bd4bf6fa2c88c48d
No such comm: 1aedc0dc276b4347bd4bf6fa2c88c48d
No such comm: 1aedc0dc276b4347bd4bf6fa2c88c48d
No such comm: 1aedc0dc276b4347bd4bf6fa2c88c48d
No such comm: 7a2a5dbb9888434eb34d0ee52e8bb993
No such comm: 7a2a5dbb9888434eb34d0ee52e8bb993
No such comm: 7a2a5dbb9888434eb34d0ee52e8bb993
No such comm: 7a2a5dbb9888434eb34d0ee52e8bb993
No such comm: 7a2a5dbb9888434eb34d0ee52e8bb993
No such comm: 7a2a5dbb9888434eb34d0ee52e8bb993
No such comm: 9b9838ee27eb492d989f050c0652fa4e
No such comm: 9b9838ee27eb492d989f050c0652fa4e
No such comm: 9b9838ee27eb492d989f050c0652fa4e
No such comm: 9b9838ee27eb492d989f050c0652fa4e
No such comm: 9b9838ee27eb492d989f050c0652fa4e
No such comm: 9b9838ee27eb492d989f050c0652fa4e
No such comm: e6a8000d261b4e9eb15fbe459af80e87
No such comm: e6a8000d261b4e9eb15fbe459af80e87
No such comm: e6a8000d261b4e9eb15fbe459af80e87
No such comm: